# Monte Carlo and Finite Difference Methods in Pricing Financial Derivatives
#### Group Number: 15 | SID: 530473808

## Imports

In [123]:
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as ss
from scipy.interpolate import interp1d
import statsmodels.api as sm
import pandas as pd

## 6. Numerical Examples

### 6.1 European Call

#### Black-Scholes Exact

In [19]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0

d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)

call_price = S0 * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
put_price  = K * np.exp(-r * T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

call_price, put_price

(0.11306156296769698, 51.324258150740974)

#### Basic Monte Carlo

In [28]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0
N = 1024

rng = np.random.default_rng(0)
Z = rng.normal(size = N)
ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
disc_payoffs = np.exp(-r*T) * np.maximum(ST - K, 0.0)
price_mc = disc_payoffs.mean()
std_err = disc_payoffs.std(ddof=1) / np.sqrt(N)

(price_mc, std_err)

(0.05913088494007343, 0.029547521295721472)

#### Antithetic Variables Monte Carlo

In [31]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0
N = 1024

rng = np.random.default_rng(0)
Z = rng.normal(size = N)
antiZ = -Z
ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
antiST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * antiZ)
disc_payoffs = np.exp(-r*T) * np.maximum(ST - K, 0.0)
antidisc_payoffs = np.exp(-r*T) * np.maximum(antiST - K, 0.0)
paired_avg = 0.5 * (disc_payoffs + antidisc_payoffs)
price_mc = np.mean(paired_avg)
std_err = paired_avg.std(ddof=1) / np.sqrt(N)

(price_mc, std_err)

(0.1451447869596705, 0.04704041466089442)

#### Importance Sampling Monte Carlo

In [36]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0
N = 1024

Z_star = (np.log(K/S0) - (r - 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
mu = max(0.0, Z_star + 1.0)

rng = np.random.default_rng(0)
Z_mu = rng.normal(loc=mu, scale=1.0, size=N)

weights = np.exp(-mu*Z_mu + 0.5*mu**2)

ST = S0 * np.exp((r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z_mu)
payoff = np.maximum(ST - K, 0.0)
disc_payoff = np.exp(-r*T) * payoff

Y = weights * disc_payoff
price_is = Y.mean()
std_err_is = Y.std(ddof=1) / np.sqrt(N)

(mu, price_is, std_err_is)

(3.321156833624894, 0.11985059471718733, 0.00414535895938793)

#### Control Variate Monte Carlo

In [39]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0
N = 1024

rng = np.random.default_rng(0)
Z  = rng.standard_normal(N)
ST = S0 * np.exp((r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z)

X = np.exp(-r*T) * np.maximum(ST - K, 0.0)
Y = np.exp(-r*T) * ST
EY = S0

cov_XY = np.cov(X, Y, ddof=1)[0,1]
var_Y  = np.var(Y, ddof=1)
beta   = cov_XY / var_Y

X_cv = X - beta*(Y - EY)

price_cv  = X_cv.mean()
se_cv     = X_cv.std(ddof=1) / np.sqrt(N)

(beta, price_cv, se_cv)

(0.013897959384434583, 0.07281122762606491, 0.028617214756624994)

#### Quasi-Monte Carlo with Sobol

In [21]:
S0, K, r, sigma, T = 70, 130, 0.07, 0.25, 1.0
N = 1024

sobol = ss.qmc.Sobol(d=1, scramble=True)
U = sobol.random(N).flatten()
Z = ss.norm.ppf(U)

ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
disc_payoffs = np.exp(-r*T) * np.maximum(ST - K, 0.0)
price_mc = disc_payoffs.mean()
std_err = disc_payoffs.std(ddof=1) / np.sqrt(N)

(price_mc, std_err)

(0.11116516164626428, 0.04739177246118439)

#### Euler-Maruyama Method

In [49]:
S0, K, r, sigma, T, npaths, nt = 100, 100, 0.07, 0.25, 1, 10000, 1000
dt = T / nt

paths_euler = []
for i in range(npaths):
    S = np.zeros(nt + 1)
    S[0] = S0
    for t in range(nt):
        change = np.random.normal()
        S[t+1] = S[t] + r * S[t] * dt + sigma * S[t] * np.sqrt(dt) * change
    paths_euler.append(S)

ST_euler = np.asarray(paths_euler)[:, -1]
payoffs_euler = np.maximum(0, ST_euler - K)
price = np.exp(-r * T) * np.mean(payoffs_euler)
price

13.518021611408617

#### Milstein Method

In [51]:
paths_milstein = []
for i in range(npaths):
    S = np.zeros(nt + 1)
    S[0] = S0
    for t in range(nt):
        change = np.random.normal()
        S[t+1] = (S[t] + r * S[t] * dt + sigma * S[t] * np.sqrt(dt) * change + 0.5 * sigma**2 * dt * (change**2 - 1) * S[t])
    paths_milstein.append(S)

ST_milstein = np.asarray(paths_milstein)[:, -1]
payoffs_milstein = np.maximum(0, ST_milstein - K)
price = np.exp(-r * T) * np.mean(payoffs_milstein)
price

13.360500360033283

#### Calculating ∆ and Γ

In [56]:
def value(S0, K = 100, r = 0.07, std = 0.25, T = 2, N = 1000, X = None):
    X = np.asarray(X)
    antiX = -X
    ST = S0*np.exp((r - 0.5*std**2)*T + std*np.sqrt(T)*X)
    antiST = S0*np.exp((r - 0.5*std**2)*T + std*np.sqrt(T)*antiX)
    payoffs = np.maximum(ST-K, 0)
    antipayoffs = np.maximum(antiST-K, 0)
    V = np.exp(-r * T) * np.mean(np.concatenate([payoffs, antipayoffs]))
    return np.mean(V)

x, S0, K, runs = 0.1, 140, 100, 100000
posArr = []
negArr = []
S0Arr = []

for _ in range(runs):
    Z = np.random.normal(size = N)
    posArr.append(value(S0+x, K, X = Z))
    negArr.append(value(S0-x, K, X = Z))
    S0Arr.append(value(S0, K, X = Z))

delta = (np.mean(posArr) - np.mean(negArr))/(2 * x)
gamma = (np.mean(posArr) - 2*np.mean(S0Arr) + np.mean(negArr))/(x**2)

delta, gamma

(0.9363030714158427, 0.0025240482194988085)

#### Heston Stochastic Volatility Model

In [23]:
def Heston_paths(N, paths, T, S0, v0, mu, rho, kappa, theta, sigma):
    dt = T / (N - 1)
    dt_sq = np.sqrt(dt)
    assert 2 * kappa * theta > sigma**2, "NOT SATISFY FELLER CONDITION!!!"
    S_T = np.zeros(paths)
    v_T = np.zeros(paths)

    for path in range(paths):
        W_S = np.random.normal(0, 1, N - 1)
        W_v = rho * W_S + np.sqrt(1 - rho**2) * np.random.normal(0, 1, N - 1)
        S = np.zeros(N)
        v = np.zeros(N)
        S[0] = S0
        v[0] = v0
        for t in range(N - 1):
            v[t+1] = abs(
                v[t] + kappa * (theta - v[t]) * dt + sigma * np.sqrt(v[t]) * dt_sq * W_v[t])
            S[t+1] = S[t] * np.exp((mu - 0.5 * v[t]) * dt + np.sqrt(v[t]) * dt_sq * W_S[t])

        S_T[path] = S[-1]
        v_T[path] = v[-1]
    return S_T, v_T

N, paths, T, S0, K, v0, r, rho, kappa, theta, sigma = 1000, 1000, 1, 100, 100, 0.04, 0.07, -0.2, 2, 0.04, 0.3
S, v = Heston_paths(N, paths, T, S0, v0, r, rho, kappa, theta, sigma)
DiscountedPayoff = np.exp(-r * T) * np.maximum(S - K, 0)
V = np.mean(DiscountedPayoff)
std_err = ss.sem(DiscountedPayoff)

V, std_err

(11.221094519900308, 0.49530868779221876)

#### Calculating Implied Volatility

In [33]:
target_price, S, K, T, r, q =20, 100, 100, 1, 0.07, 0
initial_guess = 0.5
max_iter = 10000
epsilon = 1e-8
call_put = 1
sigma = initial_guess

for _ in range(max_iter):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if call_put == 1:
        V = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        V = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    V = np.nan_to_num(V, nan=0.0)
    vega = S * np.sqrt(T) * np.exp(-q * T) * norm.pdf(d1)
    delta_price = target_price - V
    
    if abs(delta_price) < epsilon:
        break

    sigma += delta_price / vega / 100

sigma

0.4288650858948257

#### Finite Difference Methods

In [ ]:
S0 = 100
K  = 100
r  = 0.07
sigma = 0.25
T = 1
M = 400
Smax = 4.0 * S0
FORCE_N = None
SQRT_2 = math.sqrt(2.0)

def norm_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / SQRT_2))

def bs_call_price(S: float, K: float, r: float, sigma: float, T: float) -> float:
    if T <= 0:
        return max(S - K, 0.0)
    if S <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

def thomas_tridiagonal(l, d, u, b) -> np.ndarray:
    n = len(d)
    c_prime = np.zeros(n-1)
    d_prime = np.zeros(n)
    c_prime[0] = u[0] / d[0]
    d_prime[0] = b[0] / d[0]
    for i in range(1, n-1):
        denom = d[i] - l[i-1] * c_prime[i-1]
        c_prime[i] = u[i] / denom
        d_prime[i] = (b[i] - l[i-1] * d_prime[i-1]) / denom
    denom = d[n-1] - l[n-2] * c_prime[n-2]
    d_prime[n-1] = (b[n-1] - l[n-2] * d_prime[n-2]) / denom
    x = np.zeros(n)
    x[n-1] = d_prime[n-1]
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i] * x[i+1]
    return x

def setup_operator(M: int, dS: float, r: float, sigma: float):
    j = np.arange(1, M)
    S = j * dS
    a = 0.5 * sigma**2 * (S**2) / (dS**2)
    b = 0.5 * r * S / dS
    lower = (a - b)
    diag  = (-2.0 * a - r)
    upper = (a + b)
    return lower, diag, upper

def fd_solve(theta: float, S0, K, r, sigma, T, M, Smax, N=None):
    dS = Smax / M
    if N is None:
        dt_stable = (dS**2) / (sigma**2 * (Smax**2))
        N_local = max(int(math.ceil(T / min(dt_stable, T))), 2000)
    else:
        N_local = int(N)

    dt = T / N_local

    S_vals = np.linspace(0.0, Smax, M+1)
    V = np.maximum(S_vals - K, 0.0)

    lower, diag, upper = setup_operator(M, dS, r, sigma)

    I = np.ones(M-1)
    A_lower = -theta * dt * lower
    A_diag  = I - theta * dt * diag
    A_upper = -theta * dt * upper

    B_lower = (1.0 - theta) * dt * lower
    B_diag  = I + (1.0 - theta) * dt * diag
    B_upper = (1.0 - theta) * dt * upper

    start = time.perf_counter()

    for n in range(N_local, 0, -1):
        t = (n-1) * dt
        V_0 = 0.0
        V_M = Smax - K * math.exp(-r * t)

        V_int = V[1:M]

        if theta == 0.0:
            LV = lower * V[0:M-1] + diag * V[1:M] + upper * V[2:M+1]
            V_new_int = V_int + dt * LV
            V_new_int[0]  += dt * lower[0]  * V_0
            V_new_int[-1] += dt * upper[-1] * V_M
        else:
            RHS = (B_diag * V_int) + (B_lower * V[0:M-1]) + (B_upper * V[2:M+1])
            RHS[0]  -= A_lower[0]   * V_0
            RHS[-1] -= A_upper[-1]  * V_M
            V_new_int = thomas_tridiagonal(A_lower[1:], A_diag.copy(), A_upper[:-1], RHS)

        V[1:M] = V_new_int
        V[0]   = V_0
        V[M]   = V_M

    elapsed_ms = (time.perf_counter() - start) * 1000.0
    price = np.interp(S0, S_vals, V)
    return price, elapsed_ms, N_local, M
    
M_values = [200, 400, 600, 800]

results = []
price_exact = bs_call_price(S0, K, r, sigma, T)

for M in M_values:
    N_for_all = FORCE_N
    price_explicit, t_explicit, N_used, M_used = fd_solve(0.0, S0, K, r, sigma, T, M, Smax, N_for_all)
    price_implicit, t_implicit, _, _          = fd_solve(1.0, S0, K, r, sigma, T, M, Smax, N_for_all)
    price_cn, t_cn, _, _                       = fd_solve(0.5, S0, K, r, sigma, T, M, Smax, N_for_all)
    abs_err_exp = abs(price_explicit - price_exact)
    abs_err_imp = abs(price_implicit - price_exact)
    abs_err_cn  = abs(price_cn - price_exact)

    results.append({
        "N (time)": N_used,
        "M (space)": M_used,
        "BS exact": price_exact,
        "Explicit": price_explicit,
        "Implicit": price_implicit,
        "Crank–Nicolson": price_cn,
        "Explicit time (ms)": t_explicit,
        "Implicit time (ms)": t_implicit,
        "CN time (ms)": t_cn,
        "Explicit abs error": abs_err_exp,
        "Implicit abs error": abs_err_imp,
        "CN abs error": abs_err_cn,
    })

df = pd.DataFrame(results)
df

,N (time),M (space),BS exact,Explicit,Implicit,Crank–Nicolson,Explicit time (ms),Implicit time (ms),CN time (ms),Explicit abs error,Implicit abs error,CN abs error
0,2500,200,13.363881,13.356619,13.355504,13.356051,14.611083,307.809959,302.602458,0.007263,0.008378,0.007830
1,10000,400,13.363881,13.362082,13.361789,13.361926,34.312500,2596.553417,2573.483333,0.001799,0.002092,0.001956
2,22501,600,13.363881,13.363093,13.362952,13.363013,85.447666,8400.535875,8349.431875,0.000788,0.000930,0.000869
3,40000,800,13.363881,13.363447,13.363359,13.363393,211.398416,19788.383958,19750.871459,0.000435,0.000523,0.000489


### 6.2 Barrier Options

#### Down and In using Monte Carlo Random Paths

In [46]:
S0, K, B, r, sigma, T, npaths, nt = 100, 100, 90, 0.07, 0.25, 1, 1000, 1000
dt  = T / nt
sdt = np.sqrt(dt)

S = np.empty((npaths, nt + 1), dtype=float)
S[:, 0] = S0
for t in range(nt):
    Z  = np.random.normal(size=npaths)
    dW = sdt * Z
    St = S[:, t]
    S[:, t+1] = St + r*St*dt + sigma*St*dW + 0.5*(sigma**2)*St*(dW**2 - dt)

activated = (S.min(axis=1) <= B)
S_T   = S[:, -1]
payoff = np.maximum(S_T - K, 0.0)
payoff = payoff * activated.astype(float)
disc  = np.exp(-r*T)
price = disc * payoff.mean()
se    = disc * payoff.std(ddof=1) / np.sqrt(npaths)

price, se

(3.236119265610753, 0.3073866568788746)

#### Up and out using Monte Carlo Random Paths

In [51]:
S0, K, B, r, sigma, T, npaths, nt = 100, 100, 110, 0.07, 0.25, 1, 1000, 1000
dt  = T / nt
sdt = np.sqrt(dt)

S = np.empty((npaths, nt + 1), dtype=float)
S[:, 0] = S0
for t in range(nt):
    Z  = np.random.normal(size=npaths)
    dW = sdt * Z
    St = S[:, t]
    S[:, t+1] = St + r*St*dt + sigma*St*dW + 0.5*(sigma**2)*St*(dW**2 - dt)

activated = (S.max(axis=1) < B)
S_T   = S[:, -1]
payoff = np.maximum(S_T - K, 0.0)
payoff = payoff * activated.astype(float)
disc  = np.exp(-r*T)
price = disc * payoff.mean()
se    = disc * payoff.std(ddof=1) / np.sqrt(npaths)

price, se

(0.07028880801378809, 0.017016509874109527)

#### Up and In using Monte Carlo Random Paths

In [59]:
S0, K, B, r, sigma, T, npaths, nt = 100, 100, 110, 0.07, 0.25, 1, 1000, 1000
dt  = T / nt
sdt = np.sqrt(dt)

S = np.empty((npaths, nt + 1), dtype=float)
S[:, 0] = S0
for t in range(nt):
    Z  = np.random.normal(size=npaths)
    dW = sdt * Z
    St = S[:, t]
    S[:, t+1] = St + r*St*dt + sigma*St*dW + 0.5*(sigma**2)*St*(dW**2 - dt)

activated = (S.max(axis=1) >= B)
S_T   = S[:, -1]
payoff = np.maximum(S_T - K, 0.0)
payoff = payoff * activated.astype(float)
disc  = np.exp(-r*T)
price = disc * payoff.mean()
se    = disc * payoff.std(ddof=1) / np.sqrt(npaths)

price, se

(13.519347979810473, 0.5929752067116599)

#### Down and Out using Crank-Nicolson Method

In [90]:
T, S0, K, B, r, sigma = 1.0, 100.0, 90.0, 80.0, 0.07, 0.05
n_t, n_s = 400, 600

def get_result(x_data: np.array, y_data: np.array, val: float) -> float:
    li = np.where(x_data < val)[0][-1]
    ri = np.where(x_data >= val)[0][0]
    if ri == val:
        return y_data[ri]
    else:
        return (y_data[ri] - y_data[li]) / (x_data[ri] - x_data[li]) * (val - x_data[li]) + y_data[li]

def const_tri_diag_mat_solve(params: np.array, mat_b: np.array) -> np.array:
    n = len(mat_b)
    x = np.zeros(n)
    p1, p2, p3 = params[0], params[1], params[2]
    coef_A = np.zeros(n)
    coef_B = np.zeros(n)
    coef_A[1] = - p3 / p2
    coef_B[1] = mat_b[0] / p2
    alpha_gamma = p2 / p1

    for i in range(1, n - 1):
        b_gamma = mat_b[i] / p1
        coef_A[i + 1] =  - p3 / ( p1 * coef_A[i] + p2 )
        coef_B[i + 1] = ( b_gamma - coef_B[i] ) / ( coef_A [i] + alpha_gamma) 

    i = n - 1
    b_gamma = mat_b[i] / p1
    x[i] = - ( coef_B[i]  - b_gamma ) / ( coef_A[i] + alpha_gamma )
    for j in range(n - 1, 0 , -1):
        x[j - 1] = x[j] * coef_A[j] + coef_B[j]
    return x

S_max = max(4.0*K, 4.0*S0)
region = [[0, T], [B, S_max]]
right_t, left_t = sigma**2 / 2 * (T - region[0][0]), sigma**2 / 2 * (T - region[0][1])
left_x, right_x = math.log(region[1][0] / K), math.log(region[1][1] / K)
tau = abs(right_t - left_t) / n_t
h = abs(right_x - left_x) / n_x
u = np.zeros((n_t, n_x))
D =  2 * r / sigma**2

for i in range(0, n_t): 
    u[i][0] = 0
S_right = math.exp(right_x) * K  # S_max
for i in range(n_t):
    t_i = T - 2.0 * (i * tau) / (sigma**2)   # calendar time for row i
    u[i][n_x - 1] = 1.0 - (K / S_right) * math.exp(-r * (T - t_i))
for i in range(0, n_x):
    u[0][i] = max(0 , 1 - math.exp(- ( left_x + i * h ) ) )

size = n_x - 2
p1, p2, p3 = - tau, 2 * h**2 + 2 * tau + tau * h * (1 + D), - tau - tau * h * (1 + D)
A_mat_params = np.array([p1, p2, p3])
p4 = 2 * h**2 - 2 * tau - tau * h * (1 + D)

for k in range(0, n_t - 1):    
    b = np.zeros(size)
    b[0] = - p1 * u[k + 1][0] - p1 * u[k][0] + p4 * u[k][1] - p3 * u[k][2]
    b[1:size - 1] = - p1 * u[k][1:size - 1] + p4 * u[k][2:size] - p3 * u[k][3:size + 1]
    b[size - 1] = - p3 * u[k + 1][n_x - 1] - p1 * u[k][n_x - 3] + p4 * u[k][n_x - 2] - p3 * u[k][n_x - 1]
    res = const_tri_diag_mat_solve(A_mat_params, b)
    u[k + 1][1:n_x - 1] = res
    x_data = np.exp(np.linspace(left_x, right_x, n_x)) * K
for k in range(0, n_t):
    u[k] = u[k] * x_data
    
T1 = sigma**2 * T / 2
t = np.linspace(0, T1, n_t)
t = T - 2 * t / sigma**2

s = np.linspace(math.log( region[1][0] / K ), math.log( region[1][1] / K), n_x)
s = np.exp(s) * K

get_result(s, u[-1], S0)

16.0311050638976

### 6.3 American Options

#### Monte Carlo with Longstaff-Schwartz Regresssion

In [103]:
def S_euler(S0, sigma, r, T, N, M):
    dt = T/M
    paths = []
    for i in range(N):
        S = np.zeros(M+1)
        S[0] = S0
        for n in range(M):
            epsilon = np.random.normal(loc = 0, scale = 1)
            S[n+1] = S[n] + r * S[n] * dt + sigma * S[n] * np.sqrt(dt) * epsilon
        paths.append(S)
    return paths
    
def amer_option(S0, sigma, r, T, K, step, N, M):
    paths = np.array(S_euler(S0, sigma, r, T, N, M))
    dt = T / M
    final_payoffs = np.maximum(K - paths[:,-1], 0)

    for t in range(M-1, step-1, -1):
        S_t = paths[:, t]
        ITM = np.where(S_t < K)[0]
        if len(ITM) == 0:
            continue

        X = S_t[ITM]
        Y = final_payoffs[ITM] * np.e ** (-r * dt)

        #quadratic cos higher degrees overfit (i think quadratic arbitrary tho and like a quartic or smthing could work)
        # WE ARE ESTIMATING THE CONTINUATION VALUE FORMULA USING STOCK PRICE
        # WE FIT IT USING STATSMODELS OLS, can also look more deeply into results if u do summarize(results)
        X_quadratic = np.column_stack((np.ones(len(X)), X, X ** 2)) # 1st column 1s, 2nd column X, 3rd column X^2
        model = sm.OLS(Y, X_quadratic) # instantiating and fitting model, QUADRATIC REGRESSION!!
        results = model.fit() # fititng model, this bit just basically getting results
        cont_val = results.predict(X_quadratic) # predicting using polynomial regression
        
        exercise = K - X
        final_payoffs[ITM] = np.where(exercise > cont_val, exercise, final_payoffs[ITM] * np.e ** (-r * dt))
    price = np.mean(final_payoffs) * np.e ** (-r * dt)
    return price

amer_option(50, 0.25, 0.07, 1, 150, 900, 1000, 1000)

96.7743633052969

### 6.4 Yield Curve Swap Solver

#### Newton-Raphson Method

In [113]:
def swaps(tenors, par_rates, accruals, seed_D1=None):
    n = len(tenors)
    D = [None]*n
    if seed_D1 is not None:
        D[0] = seed_D1
    else:
        D[0] = math.exp(-par_rates[0] * tenors[0])

    for k in range(1, n):
        S = par_rates[k]
        alpha_k = accruals[k]
        fixed_sum_known = sum(accruals[i] * D[i] for i in range(k))

        def f(Dk):
            return S * (fixed_sum_known + alpha_k * Dk) - (1.0 - Dk)

        fprime = S * alpha_k + 1.0
        Dk = math.exp(-S * tenors[k])

        for _ in range(12):
            step = f(Dk) / fprime
            Dk -= step
            if abs(step) < 1e-12:
                break

        D[k] = Dk
    return D

tenors    = [1, 2, 3, 4, 5, 6, 7]
alphas    = [1.0]*7
par = [0.0364, 0.0358, 0.0362, 0.0368, 0.0376, 0.0385, 0.0394]
discounts = swaps(tenors, par, alphas, seed_D1=0.9850)
discounts

[0.985,
 0.931393126086117,
 0.8981148126188792,
 0.8646085145212733,
 0.8304406528129262,
 0.7957458367043726,
 0.7609881316681772]